LLM 프롬프트를 동적으로 구성하고 재사용할 수 있도록 해주는 도구
    단일입력 : 하나의 변수로 구성된 프롬프트 템플릿
    다중입력 : 둘 이상의 변수를 사용하는 템플릿
    ChatPromptTemplate 역할 기반 프롬프트 : 시스템/사용자 역할별로 프롬프트 구성 .from_message
    PartialPromptTemplate 활용 : 프롬프트 일부를 미리 고정하고 부분 포맷팅 사용(ex 시스템 메시지는 고정..)
    프롬프트 출력 및 체인 실행 : LCEL
    프롬프트 작성 팁 : 주의사항 및 모범사례

In [1]:
 # 단일 프롬프트 사용
from langchain.prompts import PromptTemplate
# 템플릿 문자열 정의
template_str = (
    "당신은 최고 수준의 마케팅 카피라이터 입니다.\n"
    "아래 제품의 매력적인 홍보문구를 100자 내외로 작성해주세요. \n\n"
    "제품명 : {product_name}"
)
# PromptTemplate 객체 생성
product_prompt = PromptTemplate.from_template(template_str)
# 템플릿에 값 채우기
formatted_prompt = product_prompt.format(product_name="이클립스")
print(formatted_prompt)

당신은 최고 수준의 마케팅 카피라이터 입니다.
아래 제품의 매력적인 홍보문구를 100자 내외로 작성해주세요. 

제품명 : 이클립스


In [2]:
 # 프롬프트 | llm -> invoke 
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o-mini', temperature=0) #홍보문구니까 temperature은 0
# Runnable 객체 생성 LCEL
chain = product_prompt | llm
result = chain.invoke({"product_name": "신라면"})
print(result.content)

"매운 맛의 정수, 신라면! 깊고 진한 육수와 쫄깃한 면발이 어우러져, 한 입에 전해지는 한국의 맛. 지금 바로 신라면으로 뜨거운 감동을 느껴보세요!"


In [3]:
# 다중 입력
# 다중 입력 템플릿 문자열 정의
multi_template_str = (
    '미래는 뉴스 기사 제목과 키워드 입니다.\n'
    '이 정보를 바탕으로 한 문단으로 구성된 간략한 요약문을 작성하세요.\n\n'
    '제목 : {title}'
    '키워드 : {keyword}'
    )
# 프롬프트 템플릿 작성
summary_prompt = PromptTemplate(template=multi_template_str, input_variables=['title','keyword'])
# 포멧팅을 통해 프롬프트 값 확인
sample_title = '인공지능 기술의 발전과 미래'
sample_keyword = '머신러닝, 딥러닝, LLM, 랭체인, 산업 혁신'
formatted_summary_prompt = summary_prompt.format(title = sample_title, keyword = sample_keyword)
print(formatted_summary_prompt)

미래는 뉴스 기사 제목과 키워드 입니다.
이 정보를 바탕으로 한 문단으로 구성된 간략한 요약문을 작성하세요.

제목 : 인공지능 기술의 발전과 미래키워드 : 머신러닝, 딥러닝, LLM, 랭체인, 산업 혁신


In [5]:
# LCEA 출력
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()
result_summary = (summary_prompt | llm | parser).invoke({
    'title' : sample_title,
    'keyword' : sample_keyword
})
result_summary

'인공지능 기술의 발전은 머신러닝과 딥러닝의 혁신을 통해 가속화되고 있으며, 특히 대규모 언어 모델(LLM)과 랭체인 기술이 주목받고 있습니다. 이러한 기술들은 다양한 산업 분야에서 혁신을 이끌어내고 있으며, 기업들은 이를 활용하여 효율성을 높이고 새로운 비즈니스 모델을 창출하고 있습니다. 앞으로 인공지능은 더욱 발전하여 우리의 삶과 산업 구조에 큰 변화를 가져올 것으로 기대됩니다.'

In [6]:
# ChatPromptTemplate 역할 기반 프롬프트
# 시스템/사용자/어시스턴트 역할 (role)
# 시스템 메시지 : 모델의 동작을 지시
# 사용자 메시지 : 실제 사용자의 입력
# 어시스턴트 메시지 : 이전 모델이 응답한 내용이 있으면 대화맥락유지를 위해 활용함

In [7]:
from langchain.prompts import ChatPromptTemplate
system_message = ('당신은 python분야의 뛰어난 전문가이자 조언자 입니다.'
                  '사용자의 프로그래밍 질문에 대해 친절하고 이해하기 쉽게 답변해 주세요')
user_message = '{question}'
chat_prompt = ChatPromptTemplate.from_messages([
    ('system',system_message),
    ('user',user_message)
])
# 템플릿을 이용해서 문장을 완성해 보세요
sample_question = '파이썬의 클래스에 대해서 설명해주세요'
format_message = chat_prompt.format_messages(question = sample_question)
print(format_message)
# 파이프라인을 이용해서 llm 호출 및 파싱
answer = (chat_prompt | llm | parser).invoke({'question' : sample_question})
print(answer)


[SystemMessage(content='당신은 python분야의 뛰어난 전문가이자 조언자 입니다.사용자의 프로그래밍 질문에 대해 친절하고 이해하기 쉽게 답변해 주세요', additional_kwargs={}, response_metadata={}), HumanMessage(content='파이썬의 클래스에 대해서 설명해주세요', additional_kwargs={}, response_metadata={})]
파이썬의 클래스는 객체 지향 프로그래밍(OOP)의 기본 개념 중 하나로, 객체를 생성하기 위한 청print을 제공합니다. 클래스는 속성과 메서드를 포함할 수 있으며, 이를 통해 데이터와 기능을 하나의 단위로 묶을 수 있습니다.

### 클래스의 기본 구조

클래스를 정의하려면 `class` 키워드를 사용합니다. 다음은 간단한 클래스의 예입니다:

```python
class Dog:
    # 생성자 메서드
    def __init__(self, name, age):
        self.name = name  # 인스턴스 변수
        self.age = age    # 인스턴스 변수

    # 메서드
    def bark(self):
        return f"{self.name} says woof!"

# 클래스의 인스턴스 생성
my_dog = Dog("Buddy", 3)

# 메서드 호출
print(my_dog.bark())  # 출력: Buddy says woof!
```

### 주요 개념

1. **생성자 (`__init__` 메서드)**:
   - 클래스의 인스턴스가 생성될 때 호출되는 특별한 메서드입니다.
   - 인스턴스 변수(속성)를 초기화하는 데 사용됩니다.

2. **인스턴스 변수**:
   - 클래스의 인스턴스에 속하는 변수입니다. `self` 키워드를 통해 접근합니다.

3. **메서드**:
   - 클래스 내에서 정의된 함수로, 인스턴스의 행동을 정의합니다.
   - 메서드는 항상 첫 번째 매개변수로 `sel

In [8]:
# ParitalPromptTemplate : 템플릿의 일부를 부분적으로 채운 새로운 템플릿
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate
role_system_template = '당신은 {role} 분야의 전문 지식인입니다. 가능한 자세히 답변해 주세요'
system_prompt = SystemMessagePromptTemplate.from_template(role_system_template)
user_prompt = HumanMessagePromptTemplate.from_template('{question}')

# chatprompttemplate 을 생성
base_chat_prompt = ChatPromptTemplate.from_messages([system_prompt, user_prompt])

partial_chat_prompt = base_chat_prompt.partial(role='주식 투자')

In [9]:
# partial로 생성된 프롬프트에 질문만 채워 프롬프트 구성
sample_question = '현재 2025년 5월 시장 상황에서 삼성전자 주식 전망은'
message = partial_chat_prompt.format_messages(question = sample_question)
print(message)

[SystemMessage(content='당신은 주식 투자 분야의 전문 지식인입니다. 가능한 자세히 답변해 주세요', additional_kwargs={}, response_metadata={}), HumanMessage(content='현재 2025년 5월 시장 상황에서 삼성전자 주식 전망은', additional_kwargs={}, response_metadata={})]


In [11]:
# LCEL - invoke
answer = (partial_chat_prompt | llm | parser).invoke({'question' : sample_question})
print(answer)

2025년 5월의 삼성전자 주식 전망을 평가하기 위해서는 여러 가지 요소를 고려해야 합니다. 다음은 삼성전자의 주식 전망에 영향을 미칠 수 있는 주요 요인들입니다.

### 1. **반도체 시장 동향**
삼성전자는 세계 최대의 반도체 제조업체 중 하나로, 메모리 반도체(DRAM, NAND 플래시)와 비메모리 반도체(시스템 반도체) 분야에서 중요한 역할을 하고 있습니다. 2025년에는 반도체 수요가 어떻게 변화할지, 특히 AI, IoT, 5G와 같은 기술 발전에 따른 수요 증가가 예상됩니다. 반도체 가격이 안정세를 보이거나 상승세를 보인다면 삼성전자의 실적에 긍정적인 영향을 미칠 것입니다.

### 2. **스마트폰 및 전자제품 시장**
삼성전자는 스마트폰, 가전제품 등 다양한 전자제품을 제조하고 있습니다. 2025년에는 새로운 기술이 적용된 스마트폰 모델과 가전제품이 출시될 것으로 예상되며, 이러한 제품들이 소비자에게 긍정적인 반응을 얻는다면 매출 증가에 기여할 수 있습니다.

### 3. **글로벌 경제 상황**
글로벌 경제의 전반적인 상황도 삼성전자의 주가에 큰 영향을 미칠 수 있습니다. 금리 인상, 인플레이션, 지정학적 리스크(예: 미중 무역 갈등) 등은 소비자 신뢰와 기업 투자에 영향을 미칠 수 있습니다. 경제가 안정세를 보인다면 소비자 지출이 증가하고, 이는 삼성전자의 매출 증가로 이어질 수 있습니다.

### 4. **경쟁 상황**
삼성전자는 애플, 화웨이, TSMC 등과 같은 강력한 경쟁자들과 경쟁하고 있습니다. 경쟁사들의 기술 발전과 시장 점유율 변화도 삼성전자의 주가에 영향을 미칠 수 있습니다. 특히, 반도체 분야에서 TSMC의 기술력 향상은 삼성전자의 시장 점유율에 도전이 될 수 있습니다.

### 5. **ESG 및 지속 가능성**
최근 기업의 사회적 책임(ESG)과 지속 가능성에 대한 관심이 높아지고 있습니다. 삼성전자가 이러한 분야에서 긍정적인 이미지를 구축하고 지속 가능한 경영을 강화한다면, 투자자들의 신뢰를 얻고 주가에 긍정적인 영향